# Lesson 6: Planning and Stock Report Generation

* In this notebook, we dive deeper into multi-agent group chats and build a custom group chat that collaborates to generate a detailed report and stocks performance over the past month
* Finishing this complex task involves planning, which is realized by including a planning agent in the group chat
* We’ll also learn how to customize speaker’s transition into group chat


## Setup

In [3]:
llm_config={"model": "gpt-4-turbo"}

## The task!

In [4]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [5]:
import autogen

Initially we need a user proxy agent that can send other agents about this initial task. `human_input mode` is set to **ALWAYS**.
So, this agent will always ask for human input when it is its turn to speak. And if the human skips the input, it will send instructions to some writer to agent to refine the blog post.


In [6]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

Since this is a complex task, we’ll need a planning agent to decompose the task to simpler, smaller tasks and send those sub-tasks to other agents to solve them.

In [7]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

We've Also added field – `description`.

The description and system message differ from each other in the following way. 

The `system message` is an instruction telling the agent only & something that only this agent needs to know.

Whereas `description` is visible to other agents. This way, we’re letting the other  agents know about the role of this agent.



In [8]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [9]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

config `last_n_messages` means that this agent will look back last 3 messages in history and find the 1st message containing the code in backward order and execute the code.


In [11]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [12]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [13]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

Chat flow -

User proxy initiates chat with chat manager. The chat manager will broadcast this to every agent.

Then chat manager will decide which agent will speak next based on their role and description, current history.


In [14]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To accomplish the task of writing a blog post about Nvidia’s stock price performance over the past month, you would need to gather the following information using Python:

1. **Historical Stock Prices for Nvidia** - Collect daily closing prices of Nvidia stock from March 23, 2024, to April 23, 2024.
2. **Analysis of Stock Price Trends** - Calculate statistical data like the maximum, minimum, average, and changes in the stock price over the specified period.
3. **Comparison with Market Trends** - Optionally compare Nvidia's performance with major market indices (e.g., S&P 500) during the same period to highlight how Nvidia's performance aligns with or diverges from the broader market.
4. **Current News or Events Impac


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
Executor (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "fetch_nvidia_alpha_vantage.py", line 15, in <module>
    recent_data = data['2024-03-23':'2024-04-23']['4. close']  # Adjust dates if necessary
                  ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pandas/core/frame.py", line 4085, in __getitem__
    return self._getitem_slice(key)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pandas/core/generic.py", line 4349, in _getitem_slice
    slobj = self.index._convert_slice_indexer(key, kind="getitem")
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 4281, in _convert_slice_indexer
    indexer = self.sl

So, during this conversation, all the agents are triggered automatically by the group chat manager and following the pre-defined roles and descriptions. The roles and descriptions need to be carefully designed to make sure they can follow the right order.

We can also add constraints on order of triggering an agent in this task using allowed or disallowed speaker transitions below.

## Add a speaker selection policy

In [15]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [16]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [17]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To accomplish the task of writing a blog post about Nvidia’s stock price performance over the past month, you would need to gather the following information using Python:

1. **Historical Stock Prices for Nvidia** - Collect daily closing prices of Nvidia stock from March 23, 2024, to April 23, 2024.
2. **Analysis of Stock Price Trends** - Calculate statistical data like the maximum, minimum, average, and changes in the stock price over the specified period.
3. **Comparison with Market Trends** - Optionally compare Nvidia's performance with major market indices (e.g., S&P 500) during the same period to highlight how Nvidia's performance aligns with or diverges from the broader market.
4. **Current News or Events Impac


--------------------------------------------------------------------------------
Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):

It appears that you've successfully planned the next steps to access Nvidia's historical stock data using an alternative approach with Alpha Vantage. Once you follow the engineer's instructions to fetch the stock data using Alpha Vantage API, you're set to proceed with analyzing the data.

Here's a reminder of the steps you need to take after retrieving the stock data:

1. **Analyze the Data**: 
   - Calculate the percentage change, maximum, minimum, and average closing prices of Nvidia's stock during the specified period.
   - Identify any significant movements or trends in the stock prices.

2. **Contextualize the Data with Market Trends**:
   - Optionally, you might want to compare Nvidia's performan

Although, we added some constraints, the overall task completion is non-linear. It still keeps the flexibility of having some agent back and forth and chiming in when they need to. So more flexibility than sequential chat.


**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.